In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [16]:
DATASET_IMPLICIT_DIMS: dict[str, int] = {
    "MNIST": 11,
    "SVHN": 14,
    "CIFAR-10": 21
}

In [18]:
dims_df = pd.DataFrame.from_dict(DATASET_IMPLICIT_DIMS, orient='index', columns=['d'])
dims_df['2d'] = 2 * dims_df['d']
dims_df['5d'] = 5 * dims_df['d']
dims_df['d*ln(d)'] = dims_df['d'] * np.log(dims_df['d'])
dims_df['d*log2(d)'] = dims_df['d'] * np.log2(dims_df['d'])
dims_df['d^1.5*ln(d)'] = dims_df['d*ln(d)'] * np.sqrt(dims_df['d'])
dims_df['d^2'] = np.pow(dims_df['d'], 2)
dims_df

,d,2d,5d,d*ln(d),d*log2(d),d^1.5*ln(d),d^2
MNIST,11,22,55,26.376848,38.053748,87.482108,121
SVHN,14,28,70,36.946803,53.302969,138.242277,196
CIFAR-10,21,42,105,63.934971,92.238666,292.986845,441


In [19]:
dims_df = dims_df.round().astype(int)
dims_df

,d,2d,5d,d*ln(d),d*log2(d),d^1.5*ln(d),d^2
MNIST,11,22,55,26,38,87,121
SVHN,14,28,70,37,53,138,196
CIFAR-10,21,42,105,64,92,293,441
